In [ ]:
#라이브러리
import re
import time
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import pdfplumber
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions

### 네이버 증권 채권분석 리포트 pdf링크, 날짜 크롤링

In [ ]:

pdf_links=[]
for page in range(198):
    names=[]
    url="https://finance.naver.com/research/debenture_list.naver?&page=%s"%(page+1)
    response=requests.get(url, page)
    html=bs(response.content, "html.parser")
    files=html.select('td[class="file"]')                 #첨부파일 링크
    dates=html.select('td[style="padding-left:5px"]')     #작성일

    #증권사 정보 리스트
    table=html.select_one("div[class='box_type_m']")
    trs=table.find_all('tr')[2:-2]
    for tr in trs:
        try:
            name=tr.find_all("td")[1]
            names.append(name.text)
        except:
            pass

    #교보증권을 제외한 모든 첨부파일 링크, 날짜 리스트 저장
    for file, date, name in zip(files, dates, names):
        if name!="교보증권":
            try:
                pdf_links.append([date.text, name, file.find("a")["href"]])
            except:
                pass


In [ ]:
pdf_df=pd.DataFrame(pdf_links)
pdf_df.to_csv("./채권보고서href.csv", index=False)

In [ ]:
pdf_df=pd.read_csv("./채권보고서href.csv")

In [ ]:
# PDF 파일 다운로드 함수
def download_pdf(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

# PDF 파일 읽기 함수
def read_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages :
            text += page.extract_text()
    return text

In [ ]:
#pdf 텍스트 추출
bond_reports=[]
for idx in range(len(pdf_df)):
    date=pdf_df.iloc[idx,0]
    pdf_url=pdf_df.iloc[idx,2]
    pdf_save_path = "./bond_report.pdf"

    download_pdf(pdf_url, pdf_save_path)
    try:
        pdf_text = read_pdf(pdf_save_path)
    except:
        pass
    soup = bs(pdf_text, 'html.parser')
    words = [x.strip() for x in str(soup).split('\n') if x != ""]
    res = []
    for word in words :
        try : float("".join([x for x in word.split(" ")]))  #숫자로만 이루어진 문장은 제외
        except : res.append(word)
    bond_reports.append([date, res])                        #네이버 증권 리포트 날짜, 내용 저장

bond_reports=pd.DataFrame(bond_reports)
bond_reports[0]=bond_reports[0].apply(lambda x: datetime.datetime.strptime(x, "%y.%m.%d"))
bond_reports.to_csv("./Bond_report_naver.csv", index=False)       #날짜, 레포트내용 담은 dataframe → csv

### 교보증권 2005년 1월~2023년 8월 보고서 크롤링(selenium)

In [ ]:

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)
driver.get


In [ ]:
#경제/채권분석 → 구분이 채권전략인 리포트 첨부파일 다운로드(selenium)
pdf_path_list=[]
for page in range(1, pages+1):
  url="https://www.iprovest.com/weblogic/RSReportServlet?scr_id=44&menuCode=13&srch_db=1&QU=&DT1=20050101&DT2=20230830&provestz=&pageNum=%s"%(page)
  response=requests.get(url, headers=header)
  soup=bs(response.content, "html.parser")
  table=soup.select_one("body > div > table")
  rows=table.select("tr")[1:]
  driver.get(url)
  for idx, row in enumerate(rows):
    if row.find_all("td")[3].text.strip()=="채권전략":
      try:
        date=row.find_all("td")[0].text       #날짜 크롤링
        driver.find_element(By.XPATH, f'/html/body/div/table/tbody/tr[{idx+1}]/td[7]/a').click()    #첨부파일 다운로드
        pdf_path=(row.find_all("td")[6].find("img")["alt"])     #파일명 크롤링
        pdf_path_list.append([date,"./"+pdf_path])              #날짜, 파일경로 저장
      except:
        pass
  time.sleep(3)

#pdf 텍스트 추출
kyobo_bond_reports=[]
for date, path in pdf_path_list:
    try:
        pdf_text = read_pdf(path)
        soup = bs(pdf_text, 'html.parser')
        words = [x.strip() for x in str(soup).split('\n') if x != ""]
        res = []
        for word in words :
            try : float("".join([x for x in word.split(" ")]))
            except : res.append(word)
        kyobo_bond_reports.append([date, res])
    except:
        pass

In [ ]:
kyobo_bond_report=pd.DataFrame(kyobo_bond_reports)
kyobo_bond_report[0]=kyobo_bond_report[0].apply(lambda x: datetime.datetime.strptime(x, "%Y/%m/%d"))
kyobo_bond_report.to_csv("./Bond_report_kyobo.csv", index=False) #교보증권 리포트 날짜, 내용 csv